1. Importamos los datos a nuestro ambiente de trabajo y revisamos que no haya huecos.
Encontramos que hay cero huecos en la base de datos. 

In [1]:
import pandas as pd #Importar librería
df=pd.read_csv("A3.1 Khan.csv") #Importar base de datos
print(df.head()) #Imprimir primeros valores para revisar
print("Cantidad de ceros en la base de datos: ")
print(df.isnull().sum().sum()) #Contamos los valores huecos

         X1        X2        X3        X4        X5        X6        X7  \
0  0.773344 -2.438405 -0.482562 -2.721135 -1.217058  0.827809  1.342604   
1 -0.078178 -2.415754  0.412772 -2.825146 -0.626236  0.054488  1.429498   
2 -0.084469 -1.649739 -0.241308 -2.875286 -0.889405 -0.027474  1.159300   
3  0.965614 -2.380547  0.625297 -1.741256 -0.845366  0.949687  1.093801   
4  0.075664 -1.728785  0.852626  0.272695 -1.841370  0.327936  1.251219   

         X8        X9       X10  ...     X2300     X2301     X2302     X2303  \
0  0.057042  0.133569  0.565427  ... -0.027474 -1.660205  0.588231 -0.463624   
1 -0.120249  0.456792  0.159053  ... -0.246284 -0.836325 -0.571284  0.034788   
2  0.015676  0.191942  0.496585  ...  0.024985 -1.059872 -0.403767 -0.678653   
3  0.819736 -0.284620  0.994732  ...  0.357115 -1.893128  0.255107  0.163309   
4  0.771450  0.030917  0.278313  ...  0.061753 -2.273998 -0.039365  0.368801   

      X2304     X2305     X2306     X2307     X2308  y  
0 -3.952845

Calculamos la diferencia de promedios entre las clases 2 y 4 para todos los genes, e imprimimos los 10 genes con la mayor diferencia de medias

In [2]:
print(df['y'].unique()) #Encontramos las clases que hay
genes = df.drop(columns=['y']) #Seleccionamos todo el df menos la clase
mean_class2 = genes[df['y'] == 2].mean() #Calculamos promedio de los genes para toda la clase 2 
mean_class4 = genes[df['y'] == 4].mean()#Calculamos promedio de los genes para toda la clase 4 
diff = (mean_class2 - mean_class4).abs() #Calculamos la diferencia 
top10 = diff.sort_values(ascending=False).head(10) #Ordenamos de mayor a menor
print("\nTop 10 genes con mayor diferencia de medias entre clases 2 y 4:")
print(top10) #Imprimimos primeros 10

[2 4 3 1]

Top 10 genes con mayor diferencia de medias entre clases 2 y 4:
X187     3.323151
X509     2.906537
X2046    2.424515
X2050    2.401783
X129     2.165185
X1645    2.065460
X1319    2.045941
X1955    2.037340
X1003    2.011337
X246     1.837830
dtype: float64


La diferencia en el promedio de expresión de cada gen entre las clases 2 y 4 indica que ciertos genes presentan valores más altos en una clase que en la otra. En términos de inferencia, esto puede ser un indicio de que dichos genes están relacionados con los factores que permiten distinguir o clasificar las muestras entre las clases 2 y 4.

2. Calculamos el estadístico t y el p-value para comparar las medias de todos los genes entre la clase 2 y la clase 4 de la base de datos. Usamos la metodología de Bonferroni, de Holm, y de Benjamini-Hochberg para corregir por múltiples pruebas e indicamos, para cada una, qué genes tienen una expresión significativamente distinta entre las clases (maneja un controlde 0.05). Usamos la función multipletests de statsmodels.stats.multitest

In [3]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
p_values = []
t_stats = []
class2 = genes[df['y'] == 2]
class4 = genes[df['y'] == 4]
for gen in genes.columns:
    t_stat, p_val = ttest_ind(class2[gen], class4[gen], equal_var=False)
    t_stats.append(t_stat)
    p_values.append(p_val)

# Crear un DataFrame con los resultados iniciales
results = pd.DataFrame({
    'gene': genes.columns,
    't_stat': t_stats,
    'p_value': p_values
})
alpha = 0.05

# Bonferroni
results['bonferroni_corrected'], results['bonferroni_p'], _, _ = multipletests(results['p_value'], alpha=alpha, method='bonferroni')
# Holm
results['holm_corrected'], results['holm_p'], _, _ = multipletests(results['p_value'], alpha=alpha, method='holm')
# Benjamini-Hochberg (FDR)
results['bh_corrected'], results['bh_p'], _, _ = multipletests(results['p_value'], alpha=alpha, method='fdr_bh')

# Filtrar genes significativos por cada método
sig_bonf = results[results['bonferroni_corrected']]
sig_holm = results[results['holm_corrected']]
sig_bh = results[results['bh_corrected']]

# Imprimir los resultados
print("Genes significativamente distintos (Bonferroni):")
print(sig_bonf[['gene', 'p_value', 'bonferroni_p']])

print("\nGenes significativamente distintos (Holm):")
print(sig_holm[['gene', 'p_value', 'holm_p']])

print("\nGenes significativamente distintos (Benjamini-Hochberg):")
print(sig_bh[['gene', 'p_value', 'bh_p']])

Genes significativamente distintos (Bonferroni):
       gene       p_value  bonferroni_p
1        X2  7.383962e-08  1.704218e-04
35      X36  7.885432e-07  1.819958e-03
66      X67  1.413077e-05  3.261383e-02
128    X129  2.516574e-09  5.808252e-06
173    X174  5.603441e-09  1.293274e-05
...     ...           ...           ...
2045  X2046  1.769295e-14  4.083533e-11
2049  X2050  4.084836e-15  9.427801e-12
2114  X2115  7.286580e-06  1.681743e-02
2145  X2146  7.001831e-10  1.616023e-06
2246  X2247  3.066047e-07  7.076436e-04

[72 rows x 3 columns]

Genes significativamente distintos (Holm):
       gene       p_value        holm_p
1        X2  7.383962e-08  1.682067e-04
35      X36  7.885432e-07  1.786050e-03
66      X67  1.413077e-05  3.168120e-02
128    X129  2.516574e-09  5.760437e-06
173    X174  5.603441e-09  1.282067e-05
...     ...           ...           ...
2045  X2046  1.769295e-14  4.072917e-11
2049  X2050  4.084836e-15  9.419631e-12
2114  X2115  7.286580e-06  1.640938e-02
2145

3. Realizamos un experimento similar, pero ahora comparando las medias de las 4 clases de labase de datos. Para lograrlo, en vez de trabajar con el estadístico t, es recomendado realizarvpruebas de análisis de varianza (ANOVA). Dicha prueba se puede realizar con la función f_oneway de scipy.stats, pero debemos revisar bien cómo se deben ingresar los datos a dicha función, necesitarermos primero estratificarlos por clase.

In [4]:
from scipy.stats import f_oneway
f_stats = []
p_values = []
classes = df['y'].unique()
groups = [genes[df['y'] == c] for c in classes]
for gen in genes.columns:
    # Extraer la expresión del gen en cada clase
    samples = [g[gen] for g in groups]
    
    # Prueba ANOVA one way
    f_stat, p_val = f_oneway(*samples)
    
    f_stats.append(f_stat)
    p_values.append(p_val)

# Crear un dataframe con los resultados
anova_results = pd.DataFrame({
    'gene': genes.columns,
    'F_stat': f_stats,
    'p_value': p_values
})
# Aplicar correcciones por pruebas múltiples
alpha = 0.05
anova_results['bonferroni_sig'], anova_results['bonferroni_p'], _, _ = multipletests(anova_results['p_value'], alpha=alpha, method='bonferroni')
anova_results['holm_sig'], anova_results['holm_p'], _, _ = multipletests(anova_results['p_value'], alpha=alpha, method='holm')
anova_results['bh_sig'], anova_results['bh_p'], _, _ = multipletests(anova_results['p_value'], alpha=alpha, method='fdr_bh')

# Mostrar los genes significativos según cada método
print("\nGenes significativamente distintos entre las 4 clases (Bonferroni):")
print(anova_results[anova_results['bonferroni_sig']][['gene', 'p_value', 'bonferroni_p']].head())

print("\nGenes significativamente distintos entre las 4 clases (Holm):")
print(anova_results[anova_results['holm_sig']][['gene', 'p_value', 'holm_p']].head())

print("\nGenes significativamente distintos entre las 4 clases (Benjamini-Hochberg):")
print(anova_results[anova_results['bh_sig']][['gene', 'p_value', 'bh_p']].head())


Genes significativamente distintos entre las 4 clases (Bonferroni):
   gene       p_value  bonferroni_p
0    X1  3.839240e-20  8.860966e-17
1    X2  1.977997e-13  4.565218e-10
2    X3  5.004749e-07  1.155096e-03
16  X17  8.650499e-07  1.996535e-03
28  X29  7.937953e-09  1.832080e-05

Genes significativamente distintos entre las 4 clases (Holm):
   gene       p_value        holm_p
0    X1  3.839240e-20  8.837930e-17
1    X2  1.977997e-13  4.456428e-10
2    X3  5.004749e-07  1.031479e-03
16  X17  8.650499e-07  1.767297e-03
28  X29  7.937953e-09  1.706660e-05

Genes significativamente distintos entre las 4 clases (Benjamini-Hochberg):
   gene       p_value          bh_p
0    X1  3.839240e-20  1.265852e-17
1    X2  1.977997e-13  8.152174e-12
2    X3  5.004749e-07  4.657645e-06
8    X9  8.500660e-05  4.036939e-04
11  X12  2.418413e-02  4.820118e-02


4. Separamos los datos en entrenamiento y prueba, construimos y entrenamos un modelo de SVM con un kernel lineal, con un kernel polinomial de orden 3, y con un kernel radial. Para evitar que el tiempo de procesamiento sea exagerado, seleccionamos solamente algunas variables, partiendo de los resultados que obtuvimos en los puntos anteriores. Esta no es una práctica adecuada, pues estamos cayendo en una situación de fuga de datos. 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
selected_genes = anova_results.sort_values('p_value').head(10)['gene']
X = df[selected_genes]
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
svm_linear = SVC(kernel='linear', random_state=42)
svm_linear.fit(X_train, y_train)
y_pred_linear = svm_linear.predict(X_test)
svm_poly = SVC(kernel='poly', degree=3, random_state=42)
svm_poly.fit(X_train, y_train)
y_pred_poly = svm_poly.predict(X_test)
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train, y_train)
y_pred_rbf = svm_rbf.predict(X_test)

5. Calculamos para los 3 modelos , las métricas que consideramos importantes para comparar los desempeños. 

In [6]:
from sklearn.metrics import classification_report, accuracy_score
print("\n SVM con kernel lineal")
print("Accuracy:", accuracy_score(y_test, y_pred_linear))
print(classification_report(y_test, y_pred_linear))
print("\n SVM con kernel polinomial (grado 3)")
print("Accuracy:", accuracy_score(y_test, y_pred_poly))
print(classification_report(y_test, y_pred_poly))
print("\n SVM con kernel radial (RBF)")
print("Accuracy:", accuracy_score(y_test, y_pred_rbf))
print(classification_report(y_test, y_pred_rbf))


 SVM con kernel lineal
Accuracy: 0.96
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      0.89      0.94         9
           3       1.00      1.00      1.00         5
           4       0.89      1.00      0.94         8

    accuracy                           0.96        25
   macro avg       0.97      0.97      0.97        25
weighted avg       0.96      0.96      0.96        25


 SVM con kernel polinomial (grado 3)
Accuracy: 0.88
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      0.89      0.94         9
           3       1.00      0.60      0.75         5
           4       0.73      1.00      0.84         8

    accuracy                           0.88        25
   macro avg       0.93      0.87      0.88        25
weighted avg       0.91      0.88      0.88        25


 SVM con kernel radial (RBF)
Accuracy

Al comparar los tres modelos de SVM, se observa que tanto el kernel lineal como el kernel radial (RBF) obtuvieron un accuracy de 0.96, mientras que el kernel polinomial de grado 3 alcanzó un accuracy ligeramente menor (0.88).

El kernel lineal y el RBF presentan además valores muy similares de precisión, recall y F1-score, lo que indica que ambos modelos logran una clasificación casi perfecta en las cuatro clases. En cambio, el modelo con kernel polinomial muestra una ligera disminución en el desempeño, particularmente en la clase 3, donde el recall baja a 0.60.

Esto sugiere que, para este conjunto de datos, la separación entre clases parece ser casi lineal, o bien, que las fronteras entre clases no requieren una transformación muy compleja del espacio de características. Por ello, tanto el modelo lineal como el radial son adecuados, mientras que el polinomial podría estar sobreajustando ligeramente o introduciendo complejidad innecesaria.

En conclusión, el SVM con kernel lineal se perfila como la mejor opción para esta tarea específica, ya que logra una alta precisión con un modelo más simple y menor costo computacional, obteniendo resultados comparables al kernel RBF.